<a href="https://colab.research.google.com/github/ShivaanshBawa/Professionalrepo/blob/main/Movie_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importing important libraries

In [ ]:
import numpy as np
import pandas as pd
import csv

In [ ]:
movies = pd.read_csv('drive/MyDrive/moviereco/movies.csv')

In [ ]:
movies.head()

In [ ]:
credits=pd.read_csv('drive/MyDrive/moviereco/credits.csv')

In [ ]:
credits.head()

In [ ]:
movies = movies.merge(credits,on = 'title')

In [ ]:
movies.info()

extracting names of actors and directors from credits.csv

In [ ]:

import ast
def str_list(x):
  x=ast.literal_eval(x)
  l=[]
  for item in x:

    l.append(item['name'])

  return l

In [ ]:
movies["cast"]=movies["cast"].apply(str_list)

In [ ]:

import ast
def str2_list(x):
  x=ast.literal_eval(x)
  l=[]
  for item in x:
    if item['job']=="Director":
      l.append(item['name'])

  return l

In [ ]:
movies["crew"]=movies["crew"].apply(str2_list)

In [ ]:
movies["cast"]

In [ ]:
movies["crew"]

In [ ]:

movies["genres"]=movies['genres'].apply(str_list)
movies["keywords"]=movies['keywords'].apply(str_list)

Finding NA values

In [ ]:
movies.isna().sum()

In [ ]:
movies.sample(5)

Removing space

In [ ]:
def join(x):
  space=[]
  for i in x:
    space.append(i.replace(" ",""))
  return space

In [ ]:
movies["cast"]=movies["cast"].apply(join)

In [ ]:
movies["cast"]=movies["cast"].apply(lambda i:i[0:3])

In [ ]:
movies["cast"]

Columns needed for recommendation

In [ ]:
new_df=pd.DataFrame(movies,columns=['id','homepage','title','cast','crew','overview','genres','keywords'])

In [ ]:
new_df.head(5)

In [ ]:
new_df['crew']=new_df['crew'].apply(join)

In [ ]:
new_df.head()

In [ ]:
new_df.isna().sum()

In [ ]:
new_df["homepage"]=new_df["homepage"].fillna(0)

In [ ]:
url=list(enumerate(new_df["homepage"]))
new_df["urls"]=0
for i in range(len(url)):
  if new_df["homepage"][i]==0:

    x=("{}/{}".format("https://www.",str("".join(list(new_df["title"][url[i][0]].split())))))

    new_df["urls"][i]=x
  else:
    new_df["urls"][i]=new_df["homepage"][i]

In [ ]:
# def link(url,i):
#   s=url
#   x="{} + {}".format("https://www.",str(new_df.iloc[[url[i][0]]].title))
#   iF s is "link"
#     return x
#   else:
#     return s
# # new_df[new_df["homepage"].fillna("{} + {}".format("https://www.",new_df["title"]]))
new_df["urls"]

In [ ]:
# new_df.iloc[[0]].title
new_df.sample(5)

To convert string into list of strings

In [ ]:

def splitting(text):
  text=str(text).split()
  return text

In [ ]:
new_df['overview']=new_df['overview'].apply(splitting)

In [ ]:
new_df['all_tags']=new_df['overview'] + new_df['genres'] + new_df['keywords'] + new_df['cast'] + new_df['crew']

In [ ]:
new_df.drop(columns=['overview','genres' ,'keywords','cast' , 'crew','homepage'],inplace=True)

In [ ]:
new_df.head()

Converting all tags in lowercase

In [ ]:
def convert_lower(text):
  l=[]
  for item in text:
    l.append(item.lower())
  return l

In [ ]:
new_df['all_tags']=new_df['all_tags'].apply(convert_lower)

In [ ]:
new_df.head()

In [ ]:
pip install nltk

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
def steming(text):
  l=[]
  for i in text:

    l.append(ps.stem(i))
  return l

In [ ]:
new_df['all_tags']=new_df['all_tags'].apply(steming)

In [ ]:
new_df.sample(5)

In [ ]:
new_df["all_tags"]=new_df["all_tags"].apply(join)

In [ ]:
new_df['all_tags'] = new_df['all_tags'].apply(lambda x: " ".join(x))

Vectorization

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english')

In [ ]:
data = cv.fit_transform(new_df['all_tags']).toarray()
data

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(data)
similarity

In [ ]:
def movie_recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:8]:


      print("{} {}".format(new_df.iloc[i[0]].title,new_df.iloc[i[0]].urls))

In [ ]:
movie_recommend("Batman")

Importing pickle

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df,open("movie_recm.pkl","wb"))
pickle.dump(similarity,open("similarity.pkl","wb"))

In [ ]:
np.array(new_df["title"])